In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns

In [2]:
%cd /home/stepan/kaggle/cars_from_auction

/home/stepan/kaggle/cars_from_auction


In [3]:
%ls

data/  models/  prepare/  result/  review/  test/  train/


In [4]:
%ls data/

processed/  sample_submission.csv  test.csv  train.csv  zipcodes.csv


In [5]:
data_path = 'data/'
data_file = 'test.csv'

In [6]:
data = pd.read_csv(data_path + data_file, index_col=0)

In [7]:
data

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price
60314,1.6,small car,2013,manual,136,swift,40000,gasoline,suzuki,0.0,30449,490.0
12566,NaN,coupé,2004,auto,333,6er,150000,gasoline,bmw,0.0,45307,670.0
17760,NaN,station wagon,2006,auto,170,e_klasse,150000,diesel,mercedes_benz,0.0,59494,460.0
8876,NaN,limousine,99,manual,101,astra,150000,gasoline,opel,NaN,25524,NaN
80392,NaN,limousine,1975,manual,54,andere,150000,diesel,mercedes_benz,0.0,70794,1110.0
...,...,...,...,...,...,...,...,...,...,...,...,...
93878,1.4,limousine,1999,manual,86,corolla,150000,gasoline,toyota,0.0,44339,110.0
99783,NaN,station wagon,2002,auto,184,3er,150000,diesel,bmw,0.0,47574,240.0
57399,NaN,small car,2005,manual,52,fox,100000,gasoline,volkswagen,0.0,50389,60.0
97106,NaN,bus,2001,manual,151,transporter,150000,diesel,volkswagen,0.0,12209,930.0


In [8]:
data.describe()

,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price
count,20086.000000,50000.000000,50000.000000,50000.000000,41773.000000,50000.000000,42594.000000
mean,1.869466,1838.633600,120.907860,125278.300000,0.092931,51196.608000,420.933230
std,0.820776,546.886892,187.784151,39509.609835,0.290339,25750.460527,727.179269
min,0.000000,0.000000,0.000000,5000.000000,0.000000,1067.000000,10.000000
25%,1.400000,1998.000000,75.000000,100000.000000,0.000000,30926.000000,100.000000
50%,1.800000,2003.000000,110.000000,150000.000000,0.000000,50170.000000,230.000000
75%,2.000000,2008.000000,150.000000,150000.000000,0.000000,71720.750000,510.000000
max,9.700000,2016.000000,16312.000000,150000.000000,1.000000,99998.000000,51990.000000


# Transform

In [9]:
def transform_year(d):
    d = d.copy()
    year = d['registration_year']
    
    d.loc[(year < 1900) & (year <= 20), 'registration_year'] += 2000
    d.loc[(year < 1900) & (year > 20), 'registration_year'] += 1900
    return d

In [10]:
def transform_power(d):
    d = d.copy()
    power = d['power']
    car_type = d['type']
    
    d.loc[power >= 500, 'power'] = np.nan
    d.loc[power < 25, 'power'] = np.nan
    return d

In [11]:
def transform_capacity(d):
    d = d.copy()
    engine = d['engine_capacity']
    d.loc[engine > 3.3, 'engine_capacity'] = np.nan
    d.loc[engine < 0.8, 'engine_capacity'] = np.nan
    return d

In [12]:
def transform_price(d):
    d = d.copy()
    d.loc[:, 'insurance_price'] = np.log(d['insurance_price'])
    if 'price' in d:
        d.loc[:, 'price'] = np.log(d['price'])
    return d

## Apply

In [13]:
transformers = [transform_year, transform_power, transform_capacity, transform_price]

In [14]:
new_data = data
for t in transformers:
    new_data = t(new_data)

In [15]:
new_data.describe()

,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price
count,19497.000000,50000.000000,45497.000000,50000.000000,41773.000000,50000.000000,42594.000000
mean,1.778325,2003.733600,128.206277,125278.300000,0.092931,51196.608000,5.479055
std,0.477522,7.299557,59.607414,39509.609835,0.290339,25750.460527,1.042473
min,0.800000,1945.000000,25.000000,5000.000000,0.000000,1067.000000,2.302585
25%,1.400000,2000.000000,85.000000,100000.000000,0.000000,30926.000000,4.605170
50%,1.800000,2004.000000,116.000000,150000.000000,0.000000,50170.000000,5.438079
75%,2.000000,2008.000000,155.000000,150000.000000,0.000000,71720.750000,6.234411
max,3.300000,2016.000000,495.000000,150000.000000,1.000000,99998.000000,10.858807


In [16]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 60314 to 478
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   engine_capacity    19497 non-null  float64
 1   type               43834 non-null  object 
 2   registration_year  50000 non-null  int64  
 3   gearbox            48021 non-null  object 
 4   power              45497 non-null  float64
 5   model              47671 non-null  object 
 6   mileage            50000 non-null  int64  
 7   fuel               46408 non-null  object 
 8   brand              50000 non-null  object 
 9   damage             41773 non-null  float64
 10  zipcode            50000 non-null  int64  
 11  insurance_price    42594 non-null  float64
dtypes: float64(4), int64(3), object(5)
memory usage: 5.0+ MB


In [17]:
new_data.to_csv(data_path + 'processed/transformed_' + data_file)